# FOOOF - Model Description

<div class="alert alert-success">
This notebook provides a more theoretical / mathematical description of the FOOOF model and parameters.
</div>

### Introduction

In FOOOF, an electrophysiological power spectrum is fit as a combination of the background process (the arythmic, background 1/f), upon which there may be putative oscillations (hereafter referred to as 'peaks') - frequency regions in which there are 'bumps', with power over and above this background process.

This formulation roughly translates to fitting the power spectrum as:

$$power\ spectrum = BG + \sum_{n=0}^{N} G_n$$

Where $BG$ is the 'background fit' and each $G_n$ is a gaussian fit to a peak, for $N$ total peaks extracted from the power spectrum.

#### Background

The background fit uses an exponential function, fit on the semilog power spectrum (linear frequencies and log10 power values). 

The exponential is of the form:

$$BG = 10^a * \frac{1}{(b + f^\chi)}$$

Or, equivalently:

$$BG = a - \log(b + f^\chi)$$

In this formulation, the 3 parameters $a$, $b$, and $\chi$ define the background, as:
- $a$ is the 'offset'
- $b$ relates to the 'knee'
- $\chi$ is the 'slope'

Note that fitting the 'knee' parameter is optional, by default the background is fit with the 'knee' parameter set to zero. 

This fits a 1/f equivalently to fitting a linear fit in log-log space. 

#### Peaks

Regions of power over above this background activity, as defined above, are considered to be putative oscillations, and fit in the model by a gaussian.

For each gaussian, $n$, with the form:

$$G_n = amp * exp (\frac{- (x - ctr)^2}{2 * wdt^2})$$

The peak is defined in terms of the $amp$, $ctr$ and $wdt$, where:
- $amp$ is the amplitude of the peak, over and above the background
- $ctr$ is the center frequency of the peak
- $wdt$ is the width of the peak

The full power spectrum fit is therefore the combination of the background fit, defined by the exponential fit, and $N$ peaks, where each $n$ is formalized as a gaussian process.

Full method details available in the paper: [LINK WHEN AVAILABLE]

## Algorithmic Description

PICTURE!

Briefly, the FOOOF model proceeds as such:
- An initial background fit is taken across the power spectrum
- This background fit is subtracted from the power spectrum, creating a flattened spectrum
- Peaks are iteratively found in this flattened spectrum
- A full peak fit is created of all peak candidates found
- The peak fit is subtracted from the original power spectrum, creating a peak-removed power spectrum
- A final background fit is taken of the peak-removed power spectrum

<div class="alert alert-info">
To step through the algorithm in more detail, with visualizations that step through the code, go [here](03-FOOOFAlgorithm.ipynb).
</div>

# TODO
FOOOF MODEL FIT PICTURE (plot)

## FOOOF Parameters

There are a number of parameters that control the FOOOF fitting algorithm. Parameters that are exposed to be set by the user are explained in detail below. 


### Controlling Peak Fits

#### peak_width_limits (Hz)

Enforced limits on the minimum and maximum widths of extracted peaks, given as a list of [minimum bandwidth, maximum bandwidth]. We recommend bandwidths at last twice the frequency resolution of the power spectrum.

Default: [0.5, 12]

### Peak Search Stopping Criteria

An iterative procedures searches for candidate peaks in the flattened spectrum. Candidate peaks are extracted in order of decreasing amplitude, until some stopping criterion is met. 

#### max_n_peaks (int)

The maximum number of peaks that can be extracted from a given power spectrum. FOOOF will halt searching for new peaks when this number is reached. Note that FOOOF extracts peaks iteratively by amplitude (over and above the background), and so this approach will extract (up to) the _n_ largest peaks. 

Default: infinite

#### peak_threshold (in units of standard deviation)

The threshold, in terms of standard deviation of the background removed power spectrum, above which a data point must pass to be considered a candidate peak. Once a candidate peak drops below this threshold, the peak search is halted (without including the most recent candidate). 

Default: 2.0

#### min_peak_amplitude (units of power - same as the input spectrum)

The minimum amplitude, above the background fit, that a peak must have to be extracted in the initial fit stage. Once a candidate peak drops below this threshold, the peak search is halted (without including the most recent candidate). Note that because this constraint is enforced during peak search, and prior to final peak fit, returned peaks are not guaranteed to surpass this value in amplitude.

Default: 0

Note: there are two different amplitude-related halting conditions for the peak searching. By default, the relative (standard-deviation based) threshold is defined, whereas the absolute threshold is set to zero (this is largely because there is no general way to set this value without knowing the scale of the data). If both are defined, both are used and the peak search will halt when a candidate peak fails to pass either the absolute, or relative threshold. 

### Background Fitting Approach

#### background_mode (string)

The fitting approach to use for the background. Options:
    - 'fixed' : fits a 1/f, without a knee parameter (with the knee parameter 'fixed' at 0)
    - 'knee' : fits the full exponential equation, including the knee parameter. (experimental)

Default: 'fixed'

<hr>
<div class="alert alert-info">
To continue with the tutorial, to a hands-on introduction to the codebase, go [here](02-FOOOF.ipynb).
</div>